In [2]:
#check if venv is used correctly
import sys
print(sys.prefix)

/Users/shubham/Documents/questions_pro_bot/ll_venv


In [23]:
os.getcwd()

'/Users/shubham/Documents/questions_pro_bot'

In [ ]:
# !pip3 freeze | grep ollama

In [ ]:
# !pip3 install langchain_community langchain_text_splitters
# !pip3 install pypdf
# !brew install libmagic
# !pip3 install python-docx
# !pip3 install sentence-transformers
# !pip3 install langchain-chroma
# !pip3 install transformers -U
# !pip3 install -qU langchain-ollama

In [ ]:
# from langchain_ollama import OllamaEmbeddings

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader # for PDFs
from docx import Document # for doc or docx 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import warnings

# from langchain_community.embeddings import OllamaEmbeddings

In [12]:
warnings.filterwarnings("ignore")
file_path = 'documents/mentalhealthmonth21.docx' # input("Enter full file path for your document (as of now supported - csv, pdf, doc or docx) : ")
file_name, file_extension = os.path.splitext(file_path)

In [13]:
if os.path.isfile(file_path) and file_extension.lower() in ['.csv', '.pdf', '.docx']:
    print("File exist and is supported.")
else :
    raise Exception("Either file don't exist or is not supported - give full file path for your document (as of now supported - csv, pdf, doc or docx).")

File exist and is supported.


In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 300)


In [ ]:
if file_extension.lower() == '.pdf':
    loaded_pdf = PyPDFLoader(file_path).load()
    splitted_text = text_splitter.split_documents(loaded_pdf)

In [15]:
if file_extension.lower() == '.docx':
    docx = Document(file_path)
    docx_text = "\n".join([p.text for p in docx.paragraphs])
    splitted_text = text_splitter.split_text(docx_text)

In [16]:
len(splitted_text) #, len(loaded_pdf)

9

In [17]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2", show_progress=True)
# embedding_function = OllamaEmbeddings(model="llama3.2:latest", show_progress=True)

In [19]:
db = Chroma.from_texts(splitted_text, embedding_function, persist_directory='chromadb_dir')
# db = Chroma.from_documents(splitted_text, embedding_function, persist_directory='chromadb_dir')
db._collection.count()

OllamaEmbeddings: 100%|███████████████████████████| 9/9 [00:07<00:00,  1.13it/s]


9

In [20]:
# results = db2.similarity_search_with_score(query="who is the queen of herbs?", k=3) # The similarity_search_with_score function in LangChain with Chroma DB returns higher scores for less relevant documents because it uses cosine distance as the scoring metric. In cosine distance, a lower score indicates a higher similarity between the query and the document. Therefore, documents with lower scores are more relevant to the query.
# results
# results[0][0].page_content
# results[0][1]

In [21]:
results = db.similarity_search_with_relevance_scores(query="What can be done for mental health awareness ?", k=3) # Return docs and relevance scores in the range [0, 1].0 is dissimilar, 1 is most similar.
results

OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:01<00:00,  1.13s/it]


[(Document(metadata={}, page_content='Quick Post 3: \nYou are not alone. Now more than ever, we need to find ways to stay connected with each other, and our community. No one should feel alone or without the information, support and help they need. Reach out to a family, friend or neighbor. The National Suicide Prevention Lifeline is available for support 24/7 at 1-800-237-8255 or text MN to 741741. #StayConnectedMN #MHAM\nQuick Post 4:'),
  -4735.620394899524),
 (Document(metadata={}, page_content='For more posts, emails, and messages, check out our #StayConnectedMN Toolkit, which focuses on mental well-being during the COVID-19 pandemic.\nEmail\nEmails can be shared throughout an organization to provide information on mental health awareness. \nTaking care of yourself during Mental Health Awareness Month\nMental health is all around us, but, what exactly is mental health? Mental health is a person’s general sense of emotional, psychological, and cognitive well-being. Everyone has men

In [ ]:
# Cosine Similarity: Measures the cosine of the angle between vectors, indicating their similarity. Higher values mean greater similarity.

# Cosine Distance: Measures the dissimilarity between vectors as the complement of the cosine similarity. Higher values mean greater dissimilarity.

# cosine_similarity(A, B) = (A . B) / (||A|| * ||B||)
# cosine_distance(A, B) = 1 - cosine_similarity(A, B)

In [1]:
import os
import shutil

dir = 'chromadb_dir'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

In [ ]:
db2 = Chroma(persist_directory='chromadb_dir', embedding_function=embedding_function)
db2._collection.count()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
contexts = []
context_str = ""
for i, result in enumerate(results) :
    if 0 <= result[1] <= 1:
        context = result[0].page_content.replace(" \n", " ")
        contexts.append(context)
        # if i == 0 : 
        #     context = "Contexts : \n\n\tA] High Relevance Context :\n" + context
        # elif i == 1 : 
        #     context = "\n\tB] Medium Relevance Context :\n" + context
        # elif i == 2 : 
        #     context = "\n\tC] Low Relevance Context :\n" + context
        context_str = context_str + "\n" + context
print(context_str)

In [ ]:
question = "what is REISHI ??"

prompt = f"""
You are chat bot who answers the Question by using the Contexts provided below.
{context_str}

Question: {question} 
DO NOT justify your answers. DO NOT give information not mentioned in the Contexts info above.

Your Answer:
"""

print(prompt)

In [ ]:
from langchain_community.llms import ollama

In [ ]:
question = "WHAT HAPPENS DURING T’AI CHI?"

prompt = f"""
* You are chat bot who answers the Question by using the Contexts given to you.

* Contexts :{context_str}

* Question you have to answer based on above Contexts : {question} 

* Your Answer:
"""

print(prompt)

In [ ]:
import requests

# API endpoint
OLLAMA_API_URL = "http://localhost:11434/api/chat"

def send_prompt_to_llama(prompt, model="llama3.2:latest"):
    """
    Send a prompt to the LLaMA model via Ollama API with streaming disabled.
    """
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False  # Disable streaming to get the full response at once
    }

    # Send request to Ollama
    response = requests.post(OLLAMA_API_URL, json=payload, headers=headers)
    
    if response.status_code == 200:
        response_data = response.json()
        return response_data["message"].get("content", "No response from model.")
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

# Example Usage
response = send_prompt_to_llama(prompt)
print("Bot Response:", response)
